# Computational Biology

The following part loads and manage all libraries etc for this Notebook

In [1]:
!pip install gensim
!pip install numpy
#!pip install matplotlib

import numpy
import gensim.downloader as api
#import matplotlib.pyplot as plt
# wv = api.load('word2vec-google-news-300')
print("Done")

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Done


### 1. Chopping a piece of the Human Genome DNA to many short Chops of DNA

We will write a code that does what the Human Genome Project did to construct a full DNA sequence of the Human genome:
First we create a DNA sequence (list of A, T, C, G's) that represent a part of the Human Genome we want to identify. Then, we will "chop" it to pieces with some overlap between them like cutting spagetti to many little pieces. This is the input the real Human Genome Project had: lot's of unidentified little pieces from the human DNA tha need to be sorted and unified to one long sequence of DNA.



In [19]:
import math
import random
import numpy as np


DemoMode = False    # If not demo mode, we run the code on much larger DNA sequence (see below)
DebugMode = False

if DemoMode:
    DNA_Size = 600
    ChopSize = 60
    MaxIterationsAllowed = 3000
else:
    DNA_Size = 1200
    ChopSize = 100
    MaxIterationsAllowed = 300000

    
overlap_min = math.ceil(ChopSize*0.2)
overlap_max = overlap_min*2
print(f'Min-Max overlap: {overlap_min} : {overlap_max}')


# This is the original, long DNA sequence, we will work with. To make it short, we generate pseude-DNA randomly.
# In real lab, we will get this long DNA sequence from real Human genome
FullDNA = random.choices(["A", "T", "G", "C"], weights = [0.2, 0.2, 0.3, 0.3], k=DNA_Size)
FullDNA = ''.join(FullDNA)

# Cut little chops from the FullDNA with some random overlap between the chops
start_index = 0
end_index = ChopSize + random.randint(overlap_min, overlap_max)
DNA_Chop = []
i=0
while end_index<len(FullDNA):
    DNA_Slice = FullDNA[start_index:end_index]
    if DebugMode:
        print(f'Piece # {i}: {len(DNA_Slice)} long: {start_index}:{end_index} {"".join(DNA_Slice)}')
    start_index = end_index - random.randint(overlap_min, overlap_max)
    end_index = start_index + ChopSize + random.randint(overlap_min, overlap_max)
    DNA_Chop.append(''.join(DNA_Slice))
    i = i + 1
    
# Special treatment to last chop of DNA which is at the end of the FullDnaSequence
if start_index<len(FullDNA):
    start_index = start_index - random.randint(overlap_min, overlap_max)
    DNA_Slice = FullDNA[start_index:]
    DNA_Chop.append(''.join(DNA_Slice))
    if DebugMode:
        print(f'Piece # {i}: {len(DNA_Slice)} long: {start_index}:{DNA_Size} {"".join(DNA_Slice)}')

print(f'\n{i+1} chops of DNA before mixing.\n')



# We now mix the pieces so they are not in the right order

Unsorted_DNA_sequence = []
Unsorted_DNA_index = ['not used'] * (i+1)
for j in range(0,i+1):
    add_i = random.randint(0,i)
    # Looking for an empty place to add the next chop
    counter = 0
    while Unsorted_DNA_index[add_i] != 'not used':
        counter = counter + 1
        add_i = add_i + 1
        if add_i>i:
            add_i = 0
        if counter>3000:
            print('!!!break because of counter!!!')
            break
    # Insert the new piece in the empty place
    Unsorted_DNA_index[add_i] = j
    if DebugMode:
        print(f'Move chop {j} to {add_i}')
        ii = input('ok?')

if DebugMode:
    print(f'\nThese are the allocations of the chops into an unsorted list of chops')
    print(Unsorted_DNA_index)

# Save the DNA Chops in the unsorted places we just randomly decided to mix them
for j in range(0,i+1):
    #print(f'Adding piece #{Unsorted_DNA_index[j]} to the Unsorted_DNA_sequence')
    Unsorted_DNA_sequence.append(DNA_Chop[Unsorted_DNA_index[j]])
    
print('\nDone.\nAn unsorted chops of DNA is ready for the reconstruction of the original DNA')

Min-Max overlap: 20 : 40

12 chops of DNA before mixing.


Done.
An unsorted chops of DNA is ready for the reconstruction of the original DNA


### 2. Reconstructing the original full-DNA sequence from the unsorted chops

We now look at the chop sand try to connect them into one long sequence which is a reconstruction of the original DNA using the overlapping parts between the pieces.

In [20]:
# Here we reconstruct the original DNA from the chopes

DebugMode = False

if DebugMode:
    print(f'\nAll chops of DNA mixed, before merging:\n')
    for j in range(0,len(Unsorted_DNA_index)):
        str = Unsorted_DNA_sequence[j]
        print(f'{j}: {str}')

if DebugMode:
    print(f'\nlen(Unsorted_DNA_sequence)={len(Unsorted_DNA_sequence)}\n')
    print(Unsorted_DNA_sequence[0])

    
Iterations = 0
while len(Unsorted_DNA_sequence)>1:
        
    Iterations = Iterations + 1
    if Iterations>MaxIterationsAllowed:
        break
        
    Examined_DNA = Unsorted_DNA_sequence[0]
    Search_Sequence = Examined_DNA[0:overlap_min-1]
    if DebugMode:
        print(f'Search_Sequence: {Search_Sequence}')
        
        if DebugMode:
            iii=input('Pause for Debugging')
    
    print(f'len(Unsorted_DNA_sequence)={len(Unsorted_DNA_sequence)}')

    for j in range(1,len(Unsorted_DNA_sequence)):
        Iterations = Iterations + 1
        if Iterations>MaxIterationsAllowed:
            if DebugMode:
                print('!!!break bacuase too many iterations!!!!')
            break
            
        found= False
        if DebugMode:        
            print(f'Looking for [{Search_Sequence}] in {j}: {Unsorted_DNA_sequence[j]}')
        if Search_Sequence in Unsorted_DNA_sequence[j]:
            found = True
            location = Unsorted_DNA_sequence[j].find(Search_Sequence)
            DNA_to_Merge = Unsorted_DNA_sequence[j]
            if DebugMode:
                print(f'Found a match at line {j}:{location}')
                print(f'len(DNA_Chop)={len(Unsorted_DNA_sequence)}')
                print(f'{j}[{location}]: {DNA_to_Merge}')

            # Merge the 2 chops to one bigger Chop & delete the redundant Chop
            Part1 = DNA_to_Merge[:location]
            Part2 = Examined_DNA
            Merged = Part1 + Part2
            if DebugMode:
                print(f'(1) {Part1}\n(2) {Part2}\n(+) {Merged}\n')
            Unsorted_DNA_sequence[j]=Merged
            Unsorted_DNA_sequence.remove(Unsorted_DNA_sequence[0])
            break
            
    if found==False:
        temp = Unsorted_DNA_sequence[0]
        index = random.randint(0, len(Unsorted_DNA_sequence)-1)
        Unsorted_DNA_sequence[0] = Unsorted_DNA_sequence[index]
        Unsorted_DNA_sequence[index] = ' '+temp
        if DebugMode:
            print(f'Swapped 0-{index}')
            print(Unsorted_DNA_sequence[0])

print(f'\nRestored DNA = {Unsorted_DNA_sequence[0].strip()}\n')
print(f'Original DNA = {FullDNA}')


len(Unsorted_DNA_sequence)=12
len(Unsorted_DNA_sequence)=11
len(Unsorted_DNA_sequence)=10
len(Unsorted_DNA_sequence)=9
len(Unsorted_DNA_sequence)=8
len(Unsorted_DNA_sequence)=7
len(Unsorted_DNA_sequence)=6
len(Unsorted_DNA_sequence)=5
len(Unsorted_DNA_sequence)=4
len(Unsorted_DNA_sequence)=3
len(Unsorted_DNA_sequence)=2

Restored DNA = TGTGCATTCTCCACCCTTGCAAGGGGTTGGTTTAGGGGATGGATCCCTCGAACCCGCCAGGTCTGTTCTGATAAAGCGGCAATTGTGGAGATCGTCCAACCGTGATTAAGCCTTAATCCTGGGTCAACATACTCTAAAACCATAGCCCGCCGCCTGGCCTAGCGCTTATCACAGAGCTATTCAGCGCGGCCTGGAGACCCCGGGCGCCAACGGTTTTCCGGCACCCGGCGCTCATAACGACGGGTCGACCTTCAGCGGCACATATCACTAAAAAGGAGTCTGATCGGTGTGCCCAGGAACATCGGCTCGTCGGTTCTGCAGCGATGGTCACACAGAGAAGATCGTAGGCGCCGGAGTACTGTCATTGGGGTTGACTAGGCTCCAGCTGGGTGCCGTAATCTTGCTGGGAGGCACGGCGCACCCGCCCTGAGCGCCCAATAGGTCGAGCGCCTAGGCCTACAGACGGTCGGCACGTTAGTCCTTCCGTCGCAGCCTAGTGGACCTCAGCGACAGCCGGACCCCCGCGACGTCGCCTGGCCCGAGCGGTCCGACCGCAGGCTACAAAAGCATTTTTCTACTCATCGCCGGACGCCACTAGCAACACTCGCCGGCCATCGCCTGTATGTGTGGCCAGGGCTCGTGGTGAAGTTCCTATGGTCCGA